# Homework 7 Template

Rixin Li & G . Besla

look for "****"

In [ ]:
'''
Homework 7
Analytic Orbit
Farah Fauzi 
'''

In [1]:
# import necessary modules
# numpy provides powerful multi-dimensional arrays to hold and manipulate data
import numpy as np
# matplotlib provides powerful functions for plotting figures
import matplotlib.pyplot as plt
# astropy provides unit system and constants for astronomical calculations
import astropy.units as u
import astropy.constants as const
# import Latex module so we can display the results with symbols
from IPython.display import Latex
%matplotlib inline

# **** import CenterOfMass to determine the COM pos/vel of M33
from centerofmass_redo import CenterOfMass

# **** import the GalaxyMass to determine the mass of M31 for each component
from GalaxyMass import ComponentMass


Separation between the MW and M31 = 769.0 kpc
Relative Velocity between the MW and M31 = 118.0 km / s
Relative Position between M33 and M31 =  201.0 kpc
Relative Velocity between M33 and M31 =  199.0 km / s
MW halo Mass: 1.975 x 10^12 Msun
MW disk Mass: 0.075 x 10^12 Msun
MW bulge Mass: 0.010 x 10^12 Msun
Total MW Mass: 2.060 x 10^12 Msun
M31 halo Mass: 1.921 x 10^12 Msun
M31 disk Mass: 0.120 x 10^12 Msun
M31 bulge Mass: 0.019 x 10^12 Msun
Total M31 Mass: 2.060 x 10^12 Msun
M33 halo Mass: 0.187 x 10^12 Msun
M33 disk Mass: 0.009 x 10^12 Msun
Total M33 Mass: 0.196 x 10^12 Msun
Total Local Group Mass: 4.316 x 10^12 Msun


# M33AnalyticOrbit

In [22]:
class M33AnalyticOrbit:
    """ Calculate the analytical orbit of M33 around M31 """
    
    def __init__(self, filename): # **** add inputs
        """ initialize the class
        input: filename of the galaxy orbit"""

        ### get the gravitational constant (the value is 4.498502151575286e-06)
        self.G = const.G.to(u.kpc**3/u.Msun/u.Gyr**2).value
        
        ### **** store the output file name
        self.fileout = filename
        
        ### get the current pos/vel of M33 
        # **** create an instance of the  CenterOfMass class for M33 
        self.M33COM = CenterOfMass("M33_000.txt", 2)
        # **** store the position VECTOR of the M33 COM (.value to get rid of units)
        self.M33_COMP = self.M33COM.COM_P(0.1).value
        # **** store the velocity VECTOR of the M33 COM (.value to get rid of units)
        self.M33_COMV = self.M33COM.COM_V(self.M33_COMP[0],self.M33_COMP[1],self.M33_COMP[2]).value
        
        ### get the current pos/vel of M31 
        # **** create an instance of the  CenterOfMass class for M31 
        self.M31COM = CenterOfMass("M31_000.txt", 2)
        # **** store the position VECTOR of the M31 COM (.value to get rid of units)
        self.M31_COMP = self.M31COM.COM_P(0.1).value
        self.x = self.M31_COMP[0]
        self.y = self.M31_COMP[1]
        self.z = self.M31_COMP[2]
        # **** store the velocity VECTOR of the M31 COM (.value to get rid of units)
        self.M31_COMV = self.M31COM.COM_V(self.M31_COMP[0],self.M31_COMP[1],self.M31_COMP[2]).value
        
        ### store the DIFFERENCE between the vectors posM33 - posM31
        # **** create two VECTORs self.r0 and self.v0 and have them be the
        # relative position and velocity VECTORS of M33
        self.r0 = np.array(self.M33_COMP) - np.array(self.M31_COMP)
        self.v0 = np.array(self.M33_COMV) - np.array(self.M31_COMV)
        
        ### get the mass of each component in M31 
        ### disk
        # **** self.rdisk = scale length (no units)
        self.rdisk = 5
        # **** self.Mdisk set with ComponentMass function. Remember to *1e12 to get the right units. Use the right ptype
        self.Mdisk = ComponentMass("M31_000.txt",2)*1e12
        ### bulge
        # **** self.rbulge = set scale length (no units)
        self.rbulge = 1
        # **** self.Mbulge  set with ComponentMass function. Remember to *1e12 to get the right units Use the right ptype
        self.Mbulge = ComponentMass("M31_000.txt",3)*1e12
        # Halo
        # **** self.rhalo = set scale length from HW5 (no units)
        self.rhalo = 61
        # **** self.Mhalo set with ComponentMass function. Remember to *1e12 to get the right units. Use the right ptype
        self.Mhalo = ComponentMass("M31_000.txt",1)*1e12
    
    
    def HernquistAccel(self, M, r_a, x, y, z): # it is easiest if you take as an input the position VECTOR 
        """ this function returns the acceleration vector from Hernquist potential
        inputs:
            M, total halo mass
            r_a, corresponding scale length
            x, y, z, magnitude of the rlative position vector
        returns:
            acceleration vector Hern"""
        
        ### **** Store the magnitude of the position vector    
        rmag = np.sqrt(x**2+y**2+z**2) 
        #r = np.array[self.x,self.y,self.z]
        
        ### *** Store the Acceleration
        HernX = - (self.G*M)/(rmag *(r_a + rmag)**2) * self.r0[0]    #follow the formula in the HW instructions
        HernY = - (self.G*M)/(rmag *(r_a + rmag)**2) * self.r0[1]
        HernZ = - (self.G*M)/(rmag *(r_a + rmag)**2) * self.r0[2]
        # NOTE: we want an acceleration VECTOR so you need to make sure that in the Hernquist equation you 
        # use  -G*M/(rmag *(ra + rmag)**2) * r --> where the last r is a VECTOR 
        
        Hern = [HernX,HernY,HernZ]
        
        return Hern
    
    ###########################################################################################
    
    def MiyamotoNagaiAccel(self, M, rd, x ,y , z):# it is easiest if you take as an input a position VECTOR  r 
        """function returns acceleration vector from a MIyamoto-Nagai profile
        inputs: 
            M is the total mass of disk
            rd is the scale length
            x, y, z is the com position
        returns:
            acceleration vector given by -G*M/((R**2 B**2)**1.5 * r)
        """
        
        rd = self.rdisk    
        
        ### Acceleration **** follow the formula in the HW instructions
        # AGAIN note that we want a VECTOR to be returned  (see Hernquist instructions)
        # this can be tricky given that the z component is different than in the x or y directions. 
        # we can deal with this by multiplying the whole thing by an extra array that accounts for the 
        # differences in the z direction:
        #  multiply the whle thing by :   np.array([1,1,ZSTUFF]) 
        # where ZSTUFF are the terms associated with the z direction

        R = np.sqrt(r[0]**2 + r[1]**2)   
        zd = rd/5
        B = rd + np.sqrt(r[2]**2 + zd**2)
            
        ax = - (self.G*Md)/(R**2 + B**2)**1.5 * r[0]   
        ay = - (self.G*Md)/(R**2 + B**2)**1.5 * r[1]     
        az = - (self.G*Md)*B*r[2]/((R**2 + B**2)**1.5 * np.sqrt(r[2]**2 + (rd/5)**2))
            
        a = np.array[ax,ay,az]
  
        return a
        # the np.array allows for a different value for the z component of the acceleration
     
    
    def M31Accel(self, x, y, z): # input should include the position vector, r
        """function sums all acceleration vectors from each galaxy component
        inputs: x, y, z center of mass position"""

        ### Call the previous functions for the halo, bulge and disk
        # **** these functions will take as inputs variable we defined in the initialization of the class like 
        # self.rdisk etc. 
        r = np.sqrt(x**2+y**2+z**2)
        aH = self.HernquistAccel(self.Mhalo, self.rhalo, r)
        aB = self.HernquistAccel(self.Mbulge, self.rbulge, r)
        aD = self.MiyamotoNagaiAccel(self.Mdisk, r)
        
        accel = np.sum(aH, aB, aD)
        
        # return the SUM of the output of the acceleration functions - this will return a VECTOR 
        return accel
    

    def LeapFrog(self, dt, x, y, z, vx, vy, vz): # take as input r and v, which are VECTORS. Assume it is ONE vector at a time
        """ function is the LeapFrog integration scheme
        inputs: 
            dt, time interval
            x, y, z center of mass position
            vx, vy, vz """
        
        r = np.sqrt(x**2+y**2+z**2)
        v = np.sqrt(vx**2+vy**2+vz**2)
        # predict the position at the next half timestep
        rhalf = r + v *dt/2 
        
        # predict the final velocity at the next timestep using the acceleration field at the rhalf position 
        vnew = v + self.M31Accel(rhalf) * dt
        
        # predict the final position using the average of the current velocity and the final velocity
        # this accounts for the fact that we don't know how the speed changes from the current timestep to the 
        # next, so we approximate it using the average expected speed over the time interval dt. 
        rnew = r + (v +vn1)*dt/2
        
        return vnew, rnew   # **** return the new position and velcoity vectors
    
    
    
    def OrbitIntegration(self, t0, dt, tmax):
        """ **** ADD COMMENTS """

        # initialize the time to the input starting time
        t = t0
        
        # initialize an empty array of size :  rows int(tmax/dt)+2  , columns 7
        orbit = np.zeros([int(tmax/dt)+2, 7])
        
        # initialize the first row of the orbit
        orbit[0] = t0, *tuple(self.r0), *tuple(self.v0)
        # this above is equivalent to 
        # orbit[0] = t0, self.r0[0], self.r0[1], self.r0[2], self.v0[0], self.v0[1], self.v0[2]
        
        
        # initialize a counter for the orbit.  
        i = 1 # since we already set the 0th values, we start the counter at 1
        
        # start the integration (advancing in time steps and computing LeapFrog at each step)
        while (t <= tmax):  # as long as t has not exceeded the maximal time 
            
            # **** advance the time by one timestep, dt
            t += dt
            # **** store the new time in the first column of the ith row
            orbit[i, 0] = t
            
            
            # ***** advance the position and velocity using the LeapFrog scheme
            # remember that LeapFrog returns a position vector and a velocity vector  
            # as an example, if a function returns three vectors you would call the function and store 
            # the variable like:     a,b,c = function(input)
            r, v = self.LeapFrog(dt, orbit[(i-1), 1:4], orbit[(i-1), 4:])
         
    
            # ****  store the new position vector into the columns with indexes 1,2,3 of the ith row of orbit
            # TIP:  if you want columns 5-7 of the Nth row of an array called A, you would write : 
            # A[n, 5:8] 
            # where the syntax is row n, start at column 5 and end BEFORE column 8
            
            
            # ****  store the new position vector into the columns with indexes 1,2,3 of the ith row of orbit
            orbit[i,1:4] = r
            
            # **** update counter i , where i is keeping track of the number of rows (i.e. the number of time steps)
            orbit[i,4:] = v 
            i += 1
        
        # write the data to a file
        np.savetxt(self.filename, orbit, fmt = "%11.3f"*7, comments='#', 
                   header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                   .format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))
        
        # there is no return function

In [23]:
M33Orbit = M33AnalyticOrbit('orbit.txt')
M33Orbit.OrbitIntegrator(0, 0.1, 10)

UnitConversionError: Can only apply 'subtract' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)